In [16]:
import hashlib
import os
from pathlib import Path

from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langchain.retrievers import ContextualCompressionRetriever, EnsembleRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langchain_chroma import Chroma
from langchain_community.document_loaders import (
    DirectoryLoader,
    UnstructuredMarkdownLoader,
)
from langchain_community.retrievers import BM25Retriever
from langchain_core.documents import Document
from langchain_core.prompts import PromptTemplate
from langchain_core.tools import tool
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
# from sentence_transformers import SentenceTransformer

load_dotenv()

from huggingface_hub import login

In [17]:
login(token=os.getenv("HUGGINGFACE_HUB_TOKEN"))

embeddings = HuggingFaceEmbeddings(model_name="deepvk/USER-bge-m3")

gpt_oss_20b = init_chat_model(
    model="openai/gpt-oss-20b:free",
    model_provider="openai",
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url="https://openrouter.ai/api/v1",
    # # прокидывание провайдер-специфичных аргументов:
    # extra_body={"top_p": 0.9}
)

In [ ]:
#  gpt_oss_120b = init_chat_model(
#     model="openai/gpt-oss-120b",
#     model_provider="openai",
#     api_key=os.getenv("OPENROUTER_API_KEY"),
#     base_url="https://openrouter.ai/api/v1",
#     # # прокидывание провайдер-специфичных аргументов:
#     # extra_body={"top_p": 0.9}
# )

In [18]:
# Параметры данных/индекса
md_folder = "/Users/sergey/Desktop/Deteiling_agent/Data/cleaned"
persist_dir = "/Users/sergey/Desktop/Deteiling_agent/Data/ChromaDB"
collection_name = "VectorDB_deepvk_USER-bge-m3"

# Открываем (или создаём пустую) коллекцию Chroma
vectordb = Chroma(
    collection_name=collection_name,
    embedding_function=embeddings,      # важно: тот же эмбеддер, что использовался при создании
    persist_directory=persist_dir,
)

def _stable_id(doc: Document, idx: int) -> str:
    """Устойчивый ID по источнику и номеру чанка (sha1 от 'source::idx')."""
    src = (doc.metadata or {}).get("source") or (doc.metadata or {}).get("file") or (doc.metadata or {}).get("path") or ""
    key = f"{str(src)}::{idx:06d}"
    return hashlib.sha1(key.encode("utf-8")).hexdigest()

def _load_and_chunk_markdown(folder: str) -> list[Document]:
    """Загрузка Markdown и чанкинг."""
    loader = DirectoryLoader(
        path=folder,
        glob="**/*.md",
        loader_cls=UnstructuredMarkdownLoader,
        loader_kwargs={"mode": "single"},
    )
    docs = loader.load()
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=1200,
        chunk_overlap=150,
        separators=["\n\n", "\n", ".", " "],
    )
    return splitter.split_documents(docs)

def _reconstruct_docs_from_chroma(db: Chroma) -> list[Document]:
    """Восстановить документы из уже существующей коллекции (на случай отсутствия исходников)."""
    raw = db._collection.get(include=["documents", "metadatas", "ids"])  # приватное API, допустимо для восстановления
    docs = []
    for txt, md in zip(raw.get("documents", []), raw.get("metadatas", [])):
        docs.append(Document(page_content=txt or "", metadata=md or {}))
    return docs

# 2) Если коллекция пуста — единовременно наполняем её
if vectordb._collection.count() == 0:
    recursive_chunks = _load_and_chunk_markdown(md_folder)
    ids, to_add = [], []
    for i, d in enumerate(recursive_chunks):
        md = dict(d.metadata or {})
        md.setdefault("source", md.get("file") or md.get("path") or "md")
        md["chunk"] = i
        to_add.append(Document(page_content=d.page_content, metadata=md))
        ids.append(_stable_id(d, i))
    vectordb.add_documents(to_add, ids=ids)
    # vectordb.persist()  # опционально; современные версии сохраняют автоматически
else:
    # Коллекция уже существует — ничего не добавляем
    pass

# 3) Готовим корпус для BM25 (не персистится). Предпочтительно — из исходников; если их нет — из Chroma.
if Path(md_folder).exists():
    recursive_chunks = _load_and_chunk_markdown(md_folder)
else:
    recursive_chunks = _reconstruct_docs_from_chroma(vectordb)


In [19]:
mmr = vectordb.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 6, "fetch_k": 40, "lambda_mult": 0.5}
)

bm25 = BM25Retriever.from_documents(recursive_chunks)

ensemble = EnsembleRetriever(
    retrievers=[mmr, bm25],
    weights=[0.6, 0.4]  # пример; можно оставить равные
)

extract_prompt = PromptTemplate.from_template(
    "You compress context for a QA system.\n"
    "Question:\n{question}\n\n"
    "Document:\n{context}\n\n"
    "Return ONLY the minimal spans from the document that answer the question.\n"
    "If nothing is relevant, return an empty string."
)

extractor = LLMChainExtractor.from_llm(llm=gpt_oss_20b, prompt=extract_prompt)

ccr = ContextualCompressionRetriever(
    base_retriever=ensemble,
    base_compressor=extractor,
)

In [20]:
# Минимальные утилиты печати
def _short(s: str, n: int = 200) -> str:
    s = (s or "").replace("\n", " ")
    return s if len(s) <= n else s[:n] + "…"

def _loc(md: dict) -> str:
    parts = []
    for k in ("source", "file", "path", "page", "chunk", "id", "doc_id"):
        if k in (md or {}):
            parts.append(f"{k}={md[k]}")
    return ", ".join(parts) if parts else "—"

def _print_docs(title: str, docs):
    print(f"\n=== {title} (n={len(docs)}) ===")
    for i, d in enumerate(docs, 1):
        meta = _loc(getattr(d, "metadata", {}) or {})
        text = _short(getattr(d, "page_content", ""))
        print(f"[{i}] {meta}\n    {text}")

# Главная функция инспекции
def inspect_extraction(query: str, **retriever_kwargs):
    # 1) ДО: что вернул ретривер (MMR+BM25 через Ensemble)
    raw_docs = ensemble.invoke(query, **retriever_kwargs)
    _print_docs("До LLMChainExtractor", raw_docs)

    # 2) ПОСЛЕ: что оставил LLMChainExtractor
    compressed = extractor.compress_documents(raw_docs, query=query)
    # опционально уберём пустые спаны
    compressed = [d for d in compressed if (d.page_content or "").strip()]
    _print_docs("После LLMChainExtractor", compressed)

    # 3) Коэффициент сжатия по символам
    before = sum(len(getattr(d, "page_content", "") or "") for d in raw_docs)
    after  = sum(len(getattr(d, "page_content", "") or "") for d in compressed)
    ratio = (after / before) if before > 0 else 0.0
    print(f"\nСжатие: {after}/{before} = {ratio:.1%}")

    return compressed  # на всякий случай возвращаем «после»

In [21]:
test_query = "Как убрать неприятный запах в салоне?"
inspect_extraction(test_query)


=== До LLMChainExtractor (n=9) ===
[1] source=/Users/sergey/Desktop/Deteiling_agent/Data/cleaned/kak-izbavitsya-ot-zapaha-v-salone-avto_cleaned.md, chunk=235
    Почему появляется запах в автомобиле?  Чем убрать неприятный запах в машине? Правила уборки
[2] source=/Users/sergey/Desktop/Deteiling_agent/Data/cleaned/khimchistka-salona-avtomobilya-svoimi-rukami_cleaned.md, chunk=435
    . Готовимся к работе 48.jpg Мы узнали, что нужно для химчистки салона авто своими руками. Но, прежде чем вы возьметесь за дело, нужно очистить пространство от всего, что не нуждается в химической обра…
[3] source=/Users/sergey/Desktop/Deteiling_agent/Data/cleaned/kak-izbavitsya-ot-zapaha-v-salone-avto_cleaned.md, chunk=244
    Нейтрализатор Air Re-Fresher Sweet Summer Breeze. Продукт эффективно работает даже со сложными “отдушками”, включая мокрую шерсть, табак или рыбу. Средство оставляет после себя легкий аромат летней св…
[4] source=/Users/sergey/Desktop/Deteiling_agent/Data/cleaned/kak-izbavitsya-ot-z

[Document(metadata={'chunk': 435, 'source': '/Users/sergey/Desktop/Deteiling_agent/Data/cleaned/khimchistka-salona-avtomobilya-svoimi-rukami_cleaned.md'}, page_content='Салон после обработки будет нуждаться в просушке. Если вы проигнорируете этот момент, то можно столкнуться с неприятным запахом или даже плесенью.'),
 Document(metadata={'chunk': 244, 'source': '/Users/sergey/Desktop/Deteiling_agent/Data/cleaned/kak-izbavitsya-ot-zapaha-v-salone-avto_cleaned.md'}, page_content='Нейтрализатор Air Re-Fresher Sweet Summer Breeze… Нейтрализатор нужно активировать при включенном кондиционере всего на 15\u202fминут.  \n\nAIRCONDITIONAR DEODORANT STEAM\u202fCarmate… Нейтрализация обеспечивается за счёт диоксида хлора.'),
 Document(metadata={'chunk': 245, 'source': '/Users/sergey/Desktop/Deteiling_agent/Data/cleaned/kak-izbavitsya-ot-zapaha-v-salone-avto_cleaned.md'}, page_content='Уничтожитель запахов Gut\u202fDuft от\u202fBorger.'),
 Document(metadata={'source': '/Users/sergey/Desktop/Deteili